In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
# from datasets import load_dataset
# from itertools import islice

# # Load in streaming mode
# dataset_stream = load_dataset(
#     "ai4bharat/Spoken-Tutorial", 
#     "indic2en", 
#     split="hindi", 
#     streaming=True
# )

# # Take first 100 samples using islice (efficient)
# first_100_samples = list(islice(dataset_stream, 100))

# # Check the first sample
# print(first_100_samples)


In [3]:
# from datasets import load_dataset

# # Load only the Hindi split from the indic2en config
# dataset = load_dataset(
#     "ai4bharat/Spoken-Tutorial",
#     "indic2en",
#     split="hindi"   # loads Hindi subset into local cache
# )

# # Take the first 100 samples
# first_100 = dataset.select(range(100))

# print(first_100[0])   # Inspect first sample


In [4]:
repo = "Ed-168/Fine-tuned-wav2vec2-BERT-indian-languages"

In [5]:
from datasets import load_dataset

common_voice_train = load_dataset(
    "mozilla-foundation/common_voice_16_0",
    "hi",
    split="train+validation",
    trust_remote_code=True
)
common_voice_test = load_dataset(
    "mozilla-foundation/common_voice_16_0",
    "hi",
    split="test",
    trust_remote_code=True
)


c:\Users\EDWIN\OneDrive\Documents\GitHub\BhasaAnuvaad-multilingual-ASR\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes" , "variant"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes" , "variant"])

In [7]:
common_voice_train[0]

{'path': 'C:\\Users\\EDWIN\\.cache\\huggingface\\datasets\\downloads\\extracted\\2c6cd998a8800b56f2fb15d7259927dbca1bd0fa2f05133309cf0f10257b5443\\hi_train_0/common_voice_hi_26008353.mp3',
 'audio': {'path': 'C:\\Users\\EDWIN\\.cache\\huggingface\\datasets\\downloads\\extracted\\2c6cd998a8800b56f2fb15d7259927dbca1bd0fa2f05133309cf0f10257b5443\\hi_train_0/common_voice_hi_26008353.mp3',
  'array': array([ 6.46234854e-26, -1.35709319e-25, -8.07793567e-26, ...,
          1.06425944e-07,  4.46417090e-08,  2.61451660e-09]),
  'sampling_rate': 48000},
 'sentence': 'हमने उसका जन्मदिन मनाया।'}

#### Displaying a random subset of the dataset

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset)
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,अगले माह शुरू होगा सेल्फ ड्राइविंग कार चलाने और कार उड़ाने का कोर्स
1,क्यों दी जाती है गहनों से ज्यादा सोने-चांदी के सिक्के खरीदने की सलाह
2,पत्नी के साथ शाइनी आहूजा दिल्ली पहुंचे
3,टॉम ने हम से झूठ बोला है।
4,हो सके तब कॉल करना।
5,"सुरक्षा में सेंध, बिना टिकट विमान में किया सफर"
6,"आते ही हिट 'रईस' का 'जालिमा', बनाया सबसे तेज एक लाख लाइक्स का रिकॉर्ड"
7,पाकिस्तान में छह तालिबान आतंकवादी ढेर
8,देखो बाहर क्या हो रहा है।
9,टिकट बुकिंग पर दिल्ली हाईकोर्ट ने जेट एयरवेज और डीजीसीए से मांगा जवाब


### Creating Wav2Vec2CTCTokenizer

#### Cleaning the transcripts
- We dont need special characters like ',' , '|' , '?' for training a speech model

In [10]:
import re
chars = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\|]'
def remove_special_chars(batch):
    batch['sentence'] = re.sub(chars , '' , batch['sentence']).lower() + ' '
    return batch

In [11]:
common_voice_train = common_voice_train.map(remove_special_chars)
common_voice_test = common_voice_test.map(remove_special_chars)

In [12]:
show_random_elements(common_voice_train.remove_columns(['path' , 'audio']))

,sentence
0,महिला सशक्तिकरण के लिये बढावा दिया जाए राज्यपाल
1,फूड एलर्जी की पहचान के लिए भी महत्वपूर्ण है गर्भनाल
2,चीनी दूतावास कहाँ है
3,क्या किसी ने कभी आपका दिल तोड़ा है
4,जानें प्राइमरी स्कूल टीचर का बेटा कैसे बना विंबलडन चैम्पियन
5,दुनिया की टॉप यूनिवर्सिटीज में भारत का न होना अखरता है राष्ट्रपति
6,विंबलडन क्वार्टर फाइनल में पहुंचे भूपति बोपन्ना
7,कांग्रेस पर भरोसा नहीं करता देशमोदी
8,मुंशी प्रेमचंद की यादगार कहानी ईदगाह
9,बच्चों के डॉक्टरों को अब भारत में मिलेगा ब्रिटिश डिप्लोमा


#### Vocabulary Creation
- extracting all the unique and useful tokens for speech recognition 

In [13]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [14]:
vocab_train = common_voice_train.map(
  extract_all_chars, batched=True, 
  batch_size=-1, keep_in_memory=True, 
  remove_columns=common_voice_train.column_names
)
vocab_test = common_voice_test.map(
  extract_all_chars, batched=True, 
  batch_size=-1, keep_in_memory=True, 
  remove_columns=common_voice_test.column_names
)

Map: 100%|██████████| 3107/3107 [00:00<00:00, 162356.45 examples/s]


In [15]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [16]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ख़': 0,
 'े': 1,
 'g': 2,
 'l': 3,
 'ा': 4,
 'ौ': 5,
 'b': 6,
 'ऋ': 7,
 'ड': 8,
 'v': 9,
 '&': 10,
 't': 11,
 'ं': 12,
 'ू': 13,
 'औ': 14,
 'c': 15,
 'थ': 16,
 'ब': 17,
 'a': 18,
 'ि': 19,
 'ह': 20,
 ' ': 21,
 'd': 22,
 'च': 23,
 'q': 24,
 'ञ': 25,
 'ड़': 26,
 'आ': 27,
 '्': 28,
 'o': 29,
 'ष': 30,
 'य': 31,
 'j': 32,
 'त': 33,
 'ई': 34,
 'ण': 35,
 'ः': 36,
 'ी': 37,
 'प': 38,
 'क': 39,
 'म': 40,
 'm': 41,
 'f': 42,
 'ै': 43,
 'ट': 44,
 'ए': 45,
 'p': 46,
 'k': 47,
 'ो': 48,
 'झ': 49,
 'ग': 50,
 'फ': 51,
 'ल': 52,
 'छ': 53,
 'w': 54,
 'i': 55,
 'फ़': 56,
 'ऊ': 57,
 '’': 58,
 'u': 59,
 'घ': 60,
 'ठ': 61,
 'ँ': 62,
 'ढ़': 63,
 'स': 64,
 'क़': 65,
 'उ': 66,
 'अ': 67,
 'e': 68,
 'ओ': 69,
 'z': 70,
 '–': 71,
 'r': 72,
 'भ': 73,
 'ॉ': 74,
 'र': 75,
 "'": 76,
 'इ': 77,
 'ख': 78,
 'ग़': 79,
 'व': 80,
 'ॅ': 81,
 'ज': 82,
 'श': 83,
 'ज़': 84,
 's': 85,
 'n': 86,
 'ऑ': 87,
 'ृ': 88,
 'द': 89,
 'x': 90,
 'ऐ': 91,
 'ढ': 92,
 'ध': 93,
 '।': 94,
 '़': 95,
 'h': 96,
 'ु': 97,
 'न': 98,
 'y': 99}

In [17]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "] 

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

In [18]:
print(f"Number of tokens: {len(vocab_dict)}")

Number of tokens: 102


In [19]:
# save the vocabulary as a json file

import json

with open("vocab.json", 'w') as vocab_file:
    json.dump(vocab_dict , vocab_file)

In [20]:
from IPython.display import JSON
import json

with open("vocab.json", "r", encoding="utf-8") as f:
    data = json.load(f)

JSON(data) 


<IPython.core.display.JSON object>

In [21]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
tokenizer

Wav2Vec2CTCTokenizer(name_or_path='./', vocab_size=102, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	100: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	101: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	102: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [22]:
tokenizer.push_to_hub("Ed-168/Fine-tuned-wav2vec2-BERT-indian-languages")

c:\Users\EDWIN\OneDrive\Documents\GitHub\BhasaAnuvaad-multilingual-ASR\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\EDWIN\.cache\huggingface\hub\models--Ed-168--Fine-tuned-wav2vec2-BERT-indian-languages. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/Ed-168/Fine-tuned-wav2vec2-BERT-indian-languages/commit/f05066de511a88747c18f9c087d5326330906c4b', commit_message='Upload tokenizer', commit_description='', oid='f05066de511a88747c18f9c087d5326330906c4b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ed-168/Fine-tuned-wav2vec2-BERT-indian-languages', endpoint='https://huggingface.co', repo_type='model', repo_id='Ed-168/Fine-tuned-wav2vec2-BERT-indian-languages'), pr_revision=None, pr_num=None)

### Create SeamlessM4TFeatureExtractor

- The role of the SeamlessM4TFeatureExtractor is to prepare the raw audio input in a format that the model can "understand". 
- It therefore maps the sequence of one-dimensional amplitude values (aka the raw audio input) to a two-dimensional matrix of log-mel spectrogram values.
- Unlike the tokenizer, the feature extractor doesn't need to be "learned" from the data, so we can load it directly from the initial model checkpoint.

In [23]:
common_voice_train[0]['audio']['array'].shape

(195264,)

In [24]:
from transformers import SeamlessM4TFeatureExtractor

feature_extractor = SeamlessM4TFeatureExtractor.from_pretrained("facebook/w2v-bert-2.0")


In [25]:
from transformers import Wav2Vec2BertProcessor

processor = Wav2Vec2BertProcessor(feature_extractor=feature_extractor , tokenizer=tokenizer)
processor.push_to_hub(repo)

CommitInfo(commit_url='https://huggingface.co/Ed-168/Fine-tuned-wav2vec2-BERT-indian-languages/commit/631f6d2d354c2d788e6a3cf16c5453e3e7c477dc', commit_message='Upload processor', commit_description='', oid='631f6d2d354c2d788e6a3cf16c5453e3e7c477dc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ed-168/Fine-tuned-wav2vec2-BERT-indian-languages', endpoint='https://huggingface.co', repo_type='model', repo_id='Ed-168/Fine-tuned-wav2vec2-BERT-indian-languages'), pr_revision=None, pr_num=None)

### Preprocess Data

In [26]:
print(common_voice_train[0]['path'])
print(common_voice_train[0]['audio'])

C:\Users\EDWIN\.cache\huggingface\datasets\downloads\extracted\2c6cd998a8800b56f2fb15d7259927dbca1bd0fa2f05133309cf0f10257b5443\hi_train_0/common_voice_hi_26008353.mp3
{'path': 'C:\\Users\\EDWIN\\.cache\\huggingface\\datasets\\downloads\\extracted\\2c6cd998a8800b56f2fb15d7259927dbca1bd0fa2f05133309cf0f10257b5443\\hi_train_0/common_voice_hi_26008353.mp3', 'array': array([ 6.46234854e-26, -1.35709319e-25, -8.07793567e-26, ...,
        1.06425944e-07,  4.46417090e-08,  2.61451660e-09]), 'sampling_rate': 48000}


- Sampling rate :  it defines how many data points of the speech signal are measured per second.  
- The sampling rate of the audio in the dataset is 48kHz , whereas the models expects 16kHz thus needs to be respampled
- we can set the correct sampling rate by using the `cast_column` method of datasets

In [27]:
from datasets import Audio

common_voice_train = common_voice_train.cast_column("audio" , Audio(sampling_rate = 16000))
common_voice_test = common_voice_test.cast_column("audio" , Audio(sampling_rate = 16000))

In [28]:
common_voice_test[0]['audio']

{'path': 'C:\\Users\\EDWIN\\.cache\\huggingface\\datasets\\downloads\\extracted\\f07556d6bad652a7d6a4c30654b2f2a3b4f5e593b2f0d1ef1929c36bf8ad252e\\hi_test_0/common_voice_hi_30196032.mp3',
 'array': array([ 9.82254278e-11, -1.45519152e-10,  3.20142135e-10, ...,
        -2.25869662e-05, -8.05291347e-06,  1.09162269e-04]),
 'sampling_rate': 16000}

In [29]:
import IPython.display as ipd
import numpy as np
import random

rand_clip = random.randint(0 , len(common_voice_train)-1)

print(common_voice_train[rand_clip]['sentence'])

ipd.Audio(data = common_voice_train[rand_clip]['audio']['array'] , autoplay=True , rate = 16000)



उसे पैसों की ज़रूरत है। 


In [30]:
rand_clip = random.randint(0 , len(common_voice_train) -1 )
print("Target text:", common_voice_train[rand_clip]["sentence"])
print("Input array shape:", common_voice_train[rand_clip]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_clip]["audio"]["sampling_rate"])

Target text: मुझे ऑपेरा से नफ़रत है। 
Input array shape: (35136,)
Sampling rate: 16000


- Now we leverage the `Wav2Vec2BertProcessor` for formatting the data required by `Wav2vec2BertforCTC` for training
- the `Wav2Vec2BertProcessor` creates a more complex representation as the raw waveform, known as Log-Mel feature extraction.

In [31]:
def prepare_dataset(batch):
    audio = batch['audio']
    batch['input_features'] = processor(audio['array'] , sampling_rate=audio['sampling_rate'])
    batch["input_length"] = len(batch['input_features'])

    batch['labels'] = processor(text = batch['sentence']).input_ids

    return batch



In [32]:
common_voice_train = common_voice_train.map(prepare_dataset , remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset , remove_columns = common_voice_test.column_names)

Map: 100%|██████████| 3107/3107 [01:21<00:00, 38.29 examples/s]


In [33]:
max_input_length_in_sec = 5.0
common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter: 100%|██████████| 7084/7084 [00:00<00:00, 142516.69 examples/s]


### Training
- Data Collator
    - Wav2Vec2-Bert has a much larger input length than the output.
    - We pad the input according to the largest sample in the batch and not the largest overall
    - Thus we use a special padding Data 
    - This is necessary because in  speech models, input and output are of different modalities meaning that they should not be treated by the same padding function.

- Evaluation Metric
    - ASR models use Word Error rate (WER) as the metric
    - we define a `compute_metrics` function

- Load a suitable pretrained weights or checkpoint and configure accordingly
- Define the training config.

In [ ]:
from dataclasses import dataclass
from typing import Optional, Union, List, Dict, Any
import torch

# @dataclass
# class DataCollatorCTCWithPadding:
#     processor: Any
#     padding: Union[bool, str] = True
#     max_length: Optional[int] = None
#     pad_to_multiple_of: Optional[int] = None

#     def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
#         # -------------------------
#         # 1. Pad input audio features
#         # -------------------------
#         input_features = [{"input_features": f["input_features"]} for f in features]
#         batch = self.processor.feature_extractor.pad(
#             input_features,
#             padding=self.padding,
#             max_length=self.max_length,
#             pad_to_multiple_of=self.pad_to_multiple_of,
#             return_tensors="pt",
#         )

#         # -------------------------
#         # 2. Handle labels (auto-detect raw text vs token IDs)
#         # -------------------------
#         first_label = features[0]["labels"]

#         if isinstance(first_label, str):
#             # Labels are raw text → tokenize
#             with self.processor.as_target_processor():
#                 labels_batch = self.processor.tokenizer(
#                     [f["labels"] for f in features],
#                     padding=self.padding,
#                     max_length=self.max_length,
#                     pad_to_multiple_of=self.pad_to_multiple_of,
#                     return_tensors="pt",
#                 )
#         else:
#             # Labels are already token IDs
#             label_features = [{"input_ids": f["labels"]} for f in features]
#             with self.processor.as_target_processor():
#                 labels_batch = self.processor.tokenizer.pad(
#                     label_features,
#                     padding=self.padding,
#                     max_length=self.max_length,
#                     pad_to_multiple_of=self.pad_to_multiple_of,
#                     return_tensors="pt",
#                 )

#         # -------------------------
#         # 3. Replace padding with -100 for CTC loss
#         # -------------------------
#         labels = labels_batch["input_ids"].masked_fill(
#             labels_batch["attention_mask"].ne(1), -100
#         )
#         batch["labels"] = labels

#         return batch


from transformers import DataCollatorWithPadding

@dataclass 
class SimpleWav2Vec2BertCollator:
    processor: Any
    
    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        # Extract and prepare input features
        input_features = []
        for f in features:
            feature = f["input_features"]
            if not isinstance(feature, torch.Tensor):
                feature = torch.tensor(feature, dtype=torch.float32)
            if feature.dim() > 1:
                feature = feature.squeeze()
            input_features.append(feature)
        
        # Manual padding for input features
        max_length = max(len(f) for f in input_features)
        padded_features = []
        attention_masks = []
        
        for feature in input_features:
            # Pad feature
            pad_length = max_length - len(feature)
            if pad_length > 0:
                padded_feature = torch.cat([feature, torch.zeros(pad_length)])
            else:
                padded_feature = feature
            padded_features.append(padded_feature)
            
            # Create attention mask
            mask = torch.ones(len(feature))
            if pad_length > 0:
                mask = torch.cat([mask, torch.zeros(pad_length)])
            attention_masks.append(mask)
        
        batch = {
            "input_features": torch.stack(padded_features),
            "attention_mask": torch.stack(attention_masks).long()
        }
        
        # Handle labels
        labels = [f["labels"] for f in features]
        if isinstance(labels[0], str):
            with self.processor.as_target_processor():
                labels_batch = self.processor.tokenizer(
                    labels,
                    padding=True,
                    return_tensors="pt",
                    truncation=True,
                )
        else:
            # Handle pre-tokenized labels
            max_label_length = max(len(label) for label in labels)
            padded_labels = []
            for label in labels:
                if isinstance(label, list):
                    label = torch.tensor(label, dtype=torch.long)
                pad_length = max_label_length - len(label)
                if pad_length > 0:
                    padded_label = torch.cat([label, torch.full((pad_length,), self.processor.tokenizer.pad_token_id)])
                else:
                    padded_label = label
                padded_labels.append(padded_label)
            
            labels_batch = {
                "input_ids": torch.stack(padded_labels),
                "attention_mask": torch.ones_like(torch.stack(padded_labels))
            }
            
            # Update attention mask for actual tokens
            for i, original_label in enumerate(labels):
                labels_batch["attention_mask"][i, len(original_label):] = 0
        
        # Replace padding with -100
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch["attention_mask"].ne(1), -100
        )
        batch["labels"] = labels
        
        return batch

TypeError: DataCollatorWithPadding.__init__() got an unexpected keyword argument 'processor'

In [ ]:
# # data_collator = DataCollatorCTCWithPadding(processor=processor)

# data_collator = SimpleDataCollatorCTCWithPadding(processor = processor , padding = True)

data_collator = DataCollatorCTCWithPadding(processor=processor, padding="longest")


ImportError: cannot import name 'DataCollatorCTCWithPadding' from 'transformers' (c:\Users\EDWIN\OneDrive\Documents\GitHub\BhasaAnuvaad-multilingual-ASR\.venv\lib\site-packages\transformers\__init__.py)

In [70]:
import evaluate

wer_metric = evaluate.load('wer')

In [71]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [72]:
from transformers import Wav2Vec2BertForCTC

model = Wav2Vec2BertForCTC.from_pretrained(
    "facebook/w2v-bert-2.0",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.0,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    add_adapter=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)


Some weights of Wav2Vec2BertForCTC were not initialized from the model checkpoint at facebook/w2v-bert-2.0 and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2_bert.adapter.layers.0.ffn.intermediate_dense.bias', 'wav2vec2_bert.adapter.layers.0.ffn.intermediate_dense.weight', 'wav2vec2_bert.adapter.layers.0.ffn.output_dense.bias', 'wav2vec2_bert.adapter.layers.0.ffn.output_dense.weight', 'wav2vec2_bert.adapter.layers.0.ffn_layer_norm.bias', 'wav2vec2_bert.adapter.layers.0.ffn_layer_norm.weight', 'wav2vec2_bert.adapter.layers.0.residual_conv.bias', 'wav2vec2_bert.adapter.layers.0.residual_conv.weight', 'wav2vec2_bert.adapter.layers.0.residual_layer_norm.bias', 'wav2vec2_bert.adapter.layers.0.residual_layer_norm.weight', 'wav2vec2_bert.adapter.layers.0.self_attn.linear_k.bias', 'wav2vec2_bert.adapter.layers.0.self_attn.linear_k.weight', 'wav2vec2_bert.adapter.layers.0.self_attn.linear_out.bias', 'wav2vec2_bert.adapter.layers.0.self_attn.linear_out.weight', 'wav2vec2_ber

In [73]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=600,
  eval_steps=300,
  logging_steps=300,
  learning_rate=5e-5,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)


In [78]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)


C:\Users\EDWIN\AppData\Local\Temp\ipykernel_37796\2066899627.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [79]:
trainer.train()

ValueError: Unable to create tensor, you should probably activate padding with 'padding=True' to have batched tensors with the same length.

In [ ]:
trainer.push_to_hub()